# Train Network

This notebook trains a Convolutional Neural Network using categorized images.

### Install Packages

This notebook requires `Pillow`, `tensorflow` and `keras`. You may uncomment and run the cell below to have Jupyter Notebook install these for you.

In [1]:
# !pip install Pillow, tensorflow, keras

### Configuration

Set the variables below to specify where the categorical images are located, which classes to train and what the filename should be for saved training data. If you would like to train all 45+ categories, set `classes = None`.

In [2]:
image_path = 'categorical'
classes = [ 'stop', 'pedestrianCrossing', 'speedLimit35', 'keepRight', 'signalAhead' ]
# Uncomment this to train all categories
# classes = None
checkpoint_path = 'weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'

In [3]:
import matplotlib.image as mpimg
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
import Image
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max',period=1)

datagen = ImageDataGenerator(rescale = 1.0 / 255.0,  
                             rotation_range=10, width_shift_range=0.2, height_shift_range=0.2,
                             zoom_range=0.2,shear_range=20,
                             validation_split=0.2)

training_set = datagen.flow_from_directory(image_path, classes=classes, target_size=(64, 64), 
                                           color_mode='grayscale', batch_size=32, shuffle=True,
                                           subset="training")
validation_set = datagen.flow_from_directory(image_path, classes=classes, target_size=(64, 64), 
                                           color_mode='grayscale', batch_size=32, shuffle=True,
                                           subset="validation")

history = model.fit_generator(training_set,
                              validation_data=validation_set,
                              epochs=10, verbose=1, callbacks=[ checkpoint ])

Found 3761 images belonging to 5 classes.
Found 939 images belonging to 5 classes.
Epoch 1/10
 60/118 [==============>...............] - ETA: 37s - loss: 1.4933 - acc: 0.3665